### 5장. 웹 크롤링을 이용한 데이터 수집

5.5 [프로젝트] 잡플래닛 기업 평점 수집하기

<u>단일 리뷰 크롤링</u>

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By

driver = webdriver.Chrome()

corp = 'SK하이닉스'
url = 'https://www.jobplanet.co.kr/companies/20561/reviews/에스케이하이닉스'
driver.get(url)

In [2]:
# 팝업창 닫기
try:
    driver.find_element(By.CLASS_NAME, 'btn_close_x_ty1').click()
except:
    pass

In [3]:
review_list = driver.find_elements(By.CLASS_NAME, 'content_wrap')
review = review_list[0]
score_list = review.find_element(By.CLASS_NAME, 'ctbody_lft')

date = review.find_elements(By.CLASS_NAME, 'txt1')[-1].get_attribute('innerHTML')
print(date)

2023. 08


In [4]:
score_list = review.find_element(By.CLASS_NAME, 'ctbody_lft')

star_score = score_list.find_element(By.CLASS_NAME, 'star_score').get_attribute('style')
print(star_score)

width: 80%;


In [5]:
# 항목별 평점
bl_score_list = score_list.find_elements(By.CLASS_NAME, 'bl_score')
welfare_salary = bl_score_list[0].get_attribute('style')
work_life_balance = bl_score_list[1].get_attribute('style')
culture = bl_score_list[2].get_attribute('style')
promotion_opp = bl_score_list[3].get_attribute('style')
management = bl_score_list[4].get_attribute('style')

print(welfare_salary, work_life_balance, culture, promotion_opp, management)

width: 60%; width: 80%; width: 80%; width: 100%; width: 20%;


In [6]:
def convert_width_to_score(width):
    width_percent = int(width.split(' ')[1][:-2])
    score = int(width_percent / 20)
    return score

In [7]:
def extract_score_from_review(review):
    # 출력 초기화
    output = []
    # 리뷰에서 평점과 관련된 부분만 가져오기
    score_list = review.find_element(By.CLASS_NAME, 'ctbody_lft')
    # 총 평점을 가져와서 추가
    star_score = score_list.find_element(By.CLASS_NAME, 'star_score').get_attribute('style')
    output.append(convert_width_to_score(star_score))
    # 항목별 평점을 가져와서 추가
    other_score_elements = score_list.find_elements(By.CLASS_NAME, 'bl_score')
    for element in other_score_elements:
        output.append(convert_width_to_score(element.get_attribute('style')))
    return output

In [8]:
print(extract_score_from_review(review))

[4, 3, 4, 4, 5, 1]


<u>전체 리뷰 크롤링</u>

전체 페이지 수 획득

In [9]:
btn_pglast = driver.find_element(By.CLASS_NAME, 'btn_pglast')
num_pages = int(btn_pglast.get_attribute('href').split('=')[1])

다음 페이지로 이동

In [10]:
import time

def get_url(driver, url):
    while True:
        try:
            driver.get(url)
            driver.implicitly_wait(10) # 페이지가 전부 로딩될 때까지 최대 10초 기다리기
            break
        except:
            driver.close()
            driver = webdriver.Chrome()
            time.sleep(60)

리뷰 수집 및 정제

In [13]:
import pandas as pd

input_data = pd.read_excel('./resources/종목별_잡플래닛url.xlsx')
data = []

for company, url in input_data[['종목명', 'URL']].values:
    # 초기 URL 들어가기
    get_url(driver, url)
    # 전체 페이지 수 계산
    btn_pglast = driver.find_element(By.CLASS_NAME, 'btn_pglast')
    num_pages = int(btn_pglast.get_attribute('href').split('=')[1])
    if num_pages > 10:
        num_pages = 10
    # 페이지 순회 시작
    for page in range(num_pages - 1):
        print('{}:{}/{}'.format(company, page+1, num_pages))
        current_url = url + '?page={}'.format(page+1)
        get_url(driver, current_url)

        # 팝업 창이 나타나면 닫기
        try:
            driver.find_element(By.CLASS_NAME, 'btn_close_x_ty1').click()
        except:
            pass

        # 현재 페이지에서 데이터 가져오기
        while True:
            time.sleep(0.5)
            try: 
                temp = []
                review_list = driver.find_elements(By.CLASS_NAME, 'content_wrap')
                for review in review_list:
                    date = review.find_elements(By.CLASS_NAME, 'txt1')[-1].get_attribute('innerHTML')
                    score = extract_score_from_review(review)
                    record = [company, date] + score  # score는 리스트임
                data += temp
                break
            except:
                driver.close()
                driver = webdriver.Chrome()
                get_url(driver, current_url)
                time.sleep(60)

data = pd.DataFrame(data, columns=['기업', '날짜', '전체평점', '승진기회및 가능성', '복지및급여', '업무와삶의균형', '사내문화', '경영진'])

삼성전자:1/1757
삼성전자:2/1757
삼성전자:3/1757
삼성전자:4/1757
삼성전자:5/1757
삼성전자:6/1757
삼성전자:7/1757
삼성전자:8/1757
삼성전자:9/1757
삼성전자:10/1757
삼성전자:11/1757
삼성전자:12/1757
삼성전자:13/1757


데이터 정제

In [ ]:
display(data.head())

In [ ]:
# 데이터 정제
data = data.groupby(['기업', '날짜'], as_index=False)[['전체평점', '승진기회및가능성', '복지및급여', '업무와삶의균형', '사내문화', '경영진']].mean()
data.sort_values(by=['기업', '날짜'], inplace=True)
display(data.head(10))

In [ ]:
data.to_csv('./data/기업월별평점.csv', index=False, encoding='utf-8')